In [5]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from src import utils, load_data
from src.models import rigl_maml as rigl
from tqdm.notebook import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.tensorboard import SummaryWriter
from collections import namedtuple
from src import utils

from rigl_torch.RigL import RigLScheduler
from    torch import optim

print(f"found gpu: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
found gpu: True


In [6]:
rigl_params = {
    "prune_strategy": {
        "dense_allocation": 0.1,
        "sparsity_distribution": "uniform",
        "delta": 100,
        "alpha": 0.3,
        "grad_accumulation_n": 1,
        "static_topo": False,
        "ignore_linear_layers": False,
        "state_dict": None
    },
    "seeds": {
        "torch": 222,
        "cuda": 222,
        "numpy": 222
    },
    "expr_id": utils.new_expr_id("lth maml"),
    "model_training_params": {
        "training_iterations": 60000, #epoch
        "n_way": 5,                        # number of classes to choose between for each task
        "k_spt": 1,                        # k shot for support set (number of examples per class per task)
        "k_qry": 15,                       # k shot for query set (number of examples per class per task)
        "imgsz": 84,                       # image size
        "imgc": 3,                         # this isn't used anywhere????? no idea what it does???? they say its supposed to be 1 or 3...
        "task_num": 4,                     # meta model batch size
        "meta_lr": 1e-3,                   # meta model learning rate
        "update_lr": 0.01,                 # task specific model learning rate
        "update_step": 5,                  # task specific model training epochs
        "update_step_test": 10,            # task specific model testing epochs
#         "optimizer": ("adam", {"lr": 0.0001}),
#         "loss_func": "cross_entropy",
        "model_name": "MAML",
        "dataset_name": "mini_imagenet",
        "layer_definitions": None
    }
}

rigl_params["model_training_params"]["layer_definitions"] = [
    ('conv2d', [32, 3, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [rigl_params["model_training_params"]["n_way"], 32 * 5 * 5]) # 32 * 5 * 5
]
utils.set_seeds(rigl_params["seeds"])

In [7]:
args = rigl_params["model_training_params"]
dataset = load_data.mini_imagenet(args)

shuffle DB :train, b:10000, 5-way, 1-shot, 15-query, resize:84
shuffle DB :val, b:100, 5-way, 1-shot, 15-query, resize:84
shuffle DB :test, b:100, 5-way, 1-shot, 15-query, resize:84


In [8]:
accs, best_model = rigl.run(dataset, rigl_params)

train epoch 0
Time 2020-11-22 17:39:45.473798
 step: 0 	training acc: [0.19333333 0.20666667 0.20666667 0.22333333 0.22333333 0.22333333]
validating model...
val acc: [0.1927 0.2365 0.2401 0.239  0.2386 0.2397 0.2404 0.2407 0.2408 0.2402
 0.2412]
 step: 30 	training acc: [0.19333333 0.30333333 0.33666667 0.33666667 0.35666667 0.35666667]
 step: 60 	training acc: [0.19       0.32333333 0.34       0.33666667 0.34       0.34      ]
 step: 90 	training acc: [0.22333333 0.33666667 0.37       0.37       0.37       0.37666667]
 step: 120 	training acc: [0.23333333 0.32333333 0.36666667 0.38       0.38333333 0.39666667]
 step: 150 	training acc: [0.22       0.27333333 0.3        0.30666667 0.30666667 0.30666667]
 step: 180 	training acc: [0.19666667 0.21666667 0.24       0.25       0.25333333 0.25666667]
 step: 210 	training acc: [0.19333333 0.37333333 0.39666667 0.40333333 0.40666667 0.41      ]
 step: 240 	training acc: [0.20666667 0.29       0.31333333 0.31333333 0.31333333 0.31333333]
 ste